# Time Series Decomposition: When Simple Methods Break Down (And What to Do About It)

*Part 2 of 8: Advanced Decomposition Techniques & STL*

---

**Welcome back to our Time Series Analysis series!** In Part 1, we met Sarah and learned about the four fundamental components of time series. We explored basic decomposition and built our first forecast model.

But what happens when your data doesn't play by the rules?

## The Problem: When Classical Methods Fail

It's Monday morning. Marcus, a senior analyst at a global e-commerce platform, is reviewing his seasonal decomposition from last week. The model looks clean—perfect sinusoidal seasonal patterns, smooth trend line, small residuals.

There's just one problem: **it's completely wrong**.

Marcus's data has two seasonal patterns:
- **Weekly**: People shop more on weekends
- **Yearly**: Holiday shopping spikes

Classical decomposition can only handle **one** seasonal period at a time. His Black Friday forecast? Off by 40%.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose, STL, MSTL

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (16, 10)

np.random.seed(42)

## Simulating Multi-Seasonal Data

Let's create a realistic time series with multiple seasonal patterns:

In [ ]:
# Generate 3 years of hourly data
hours = pd.date_range('2021-01-01', '2023-12-31 23:00:00', freq='h')
n = len(hours)

# Components
# 1. Trend: gradual growth
trend = 100 + np.linspace(0, 50, n) + 5 * np.sin(np.linspace(0, 4*np.pi, n))

# 2. Daily seasonality (24-hour pattern)
hour_of_day = hours.hour
daily_seasonal = 15 * np.sin(2 * np.pi * hour_of_day / 24)

# 3. Weekly seasonality (7-day pattern)
day_of_week = hours.dayofweek
weekly_seasonal = 10 * (day_of_week >= 5)  # Weekend spike

# 4. Yearly seasonality (365-day pattern)
day_of_year = hours.dayofyear
yearly_seasonal = 20 * np.sin(2 * np.pi * day_of_year / 365.25)

# 5. Noise
noise = np.random.normal(0, 5, n)

# Combine
data = trend + daily_seasonal + weekly_seasonal + yearly_seasonal + noise

df = pd.DataFrame({
    'value': data,
    'trend': trend,
    'daily': daily_seasonal,
    'weekly': weekly_seasonal,
    'yearly': yearly_seasonal
}, index=hours)

print(f"Data shape: {df.shape}")
print(f"Date range: {df.index.min()} to {df.index.max()}")

In [ ]:
# Plot one month sample
sample = df.loc['2021-01-01':'2021-01-31']

plt.figure(figsize=(16, 6))
plt.plot(sample.index, sample['value'], linewidth=1.5, alpha=0.8)
plt.title('Data with Multiple Seasonalities - January 2021', fontsize=14, fontweight='bold')
plt.ylabel('Value')
plt.xlabel('Date')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Classical Decomposition: What Goes Wrong

![Multiple Seasonality Problem](multiple_seasonality_problem.png)

Let's try classical decomposition with period=24 (daily):

In [ ]:
# Classical decomposition (only captures daily pattern)
decomp_daily = seasonal_decompose(df['value'], model='additive', period=24)

# Visualize
fig, axes = plt.subplots(4, 1, figsize=(16, 12))

sample = slice('2021-01-01', '2021-01-31')

axes[0].plot(df.loc[sample].index, df.loc[sample, 'value'])
axes[0].set_title('Original Data')

axes[1].plot(df.loc[sample].index, decomp_daily.trend[sample])
axes[1].set_title('Trend')

axes[2].plot(df.loc[sample].index, decomp_daily.seasonal[sample])
axes[2].set_title('Seasonal (only daily captured)')

axes[3].plot(df.loc[sample].index, decomp_daily.resid[sample])
axes[3].set_title(f'Residuals (std={decomp_daily.resid.std():.2f}) - Contains weekly & yearly!')
axes[3].axhline(y=0, color='r', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n⚠️  Residual std: {decomp_daily.resid.std():.2f}")
print("Problem: Weekly and yearly patterns stuck in residuals!")

## Enter STL: Seasonal and Trend decomposition using Loess

![Classical vs STL](classical_vs_stl_outliers.png)

STL is more robust and flexible:

**Advantages:**
- Handles outliers better (uses LOESS instead of moving averages)
- Allows seasonal variation over time
- More robust to missing data
- Adapts to local patterns

In [ ]:
# STL decomposition
stl = STL(df['value'], 
          seasonal=25,    # Daily seasonality (24 + 1)
          trend=None,     # Auto-select
          robust=True)    # Robust to outliers

stl_result = stl.fit()

# Visualize
fig, axes = plt.subplots(4, 1, figsize=(16, 12))

sample = slice('2021-01-01', '2021-01-31')

axes[0].plot(df.loc[sample].index, df.loc[sample, 'value'], color='#06A77D')
axes[0].set_title('Original Time Series')

axes[1].plot(df.loc[sample].index, stl_result.trend[sample], color='#A23B72', linewidth=2.5)
axes[1].set_title('Trend (STL)')

axes[2].plot(df.loc[sample].index, stl_result.seasonal[sample], color='#F18F01')
axes[2].set_title('Seasonal (STL - Daily Pattern)')

axes[3].plot(df.loc[sample].index, stl_result.resid[sample], color='#C73E1D', alpha=0.7)
axes[3].axhline(y=0, color='black', linestyle='--', alpha=0.3)
axes[3].set_title(f'Residuals (STL - std={stl_result.resid.std():.2f})')

plt.tight_layout()
plt.show()

print(f"\nSTL residual std: {stl_result.resid.std():.2f}")
print(f"Classical residual std: {decomp_daily.resid.std():.2f}")
print(f"Improvement: {(1 - stl_result.resid.std()/decomp_daily.resid.std())*100:.1f}%")

## MSTL: Handling Multiple Seasonalities

![MSTL Sequential Process](mstl_sequential_concept.png)

For multiple seasonal patterns, we use **MSTL** (Multiple Seasonal-Trend decomposition).

**How it works:**
1. Extract first seasonal pattern (e.g., daily)
2. From what remains, extract second pattern (e.g., weekly)
3. Continue for all patterns
4. What's left is trend + residual

In [ ]:
# MSTL with multiple periods
mstl = MSTL(df['value'], 
            periods=(24, 24*7, int(24*365.25)),  # daily, weekly, yearly
            windows=None,  # Auto-select
            iterate=2)

mstl_result = mstl.fit()

# Visualize
fig, axes = plt.subplots(6, 1, figsize=(16, 16))

sample_3m = slice('2021-01-01', '2021-03-31')

axes[0].plot(df.loc[sample_3m].index, df.loc[sample_3m, 'value'], color='#06A77D')
axes[0].set_title('Original Time Series (Q1 2021)')

axes[1].plot(df.loc[sample_3m].index, mstl_result.trend[sample_3m], color='#A23B72', linewidth=2.5)
axes[1].set_title('Trend Component')

axes[2].plot(df.loc[sample_3m].index, mstl_result.seasonal[sample_3m].iloc[:, 0], 
            color='#2E86AB', alpha=0.7)
axes[2].set_title('Daily Seasonal (24-hour)')

axes[3].plot(df.loc[sample_3m].index, mstl_result.seasonal[sample_3m].iloc[:, 1], 
            color='#F18F01')
axes[3].set_title('Weekly Seasonal (7-day)')

axes[4].plot(df.loc[sample_3m].index, mstl_result.seasonal[sample_3m].iloc[:, 2], 
            color='#E63946', linewidth=2)
axes[4].set_title('Yearly Seasonal (365-day)')

axes[5].plot(df.loc[sample_3m].index, mstl_result.resid[sample_3m], 
            color='#C73E1D', alpha=0.7)
axes[5].axhline(y=0, color='black', linestyle='--', alpha=0.3)
axes[5].set_title(f'Residuals (std={mstl_result.resid.std():.2f})')

for ax in axes:
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n🎯 MSTL residual std: {mstl_result.resid.std():.2f}")
print("All three seasonal patterns successfully extracted!")

## Tuning STL Parameters

### Key Parameters:

**1. `seasonal`** - Controls smoothness of seasonal component
- Small values: More variation allowed
- Large values: Smoother, more stable
- Rule of thumb: period + 10

**2. `trend`** - Controls smoothness of trend
- None: Auto-select (recommended)
- Small: Responds quickly to changes
- Large: Very smooth trend

**3. `robust`** - Handle outliers
- True: Robust to outliers (recommended for real data)
- False: All points weighted equally

In [ ]:
# Compare different seasonal window sizes
fig, axes = plt.subplots(3, 1, figsize=(16, 9))

for i, seasonal_window in enumerate([7, 13, 25]):
    stl_temp = STL(df['value'], seasonal=seasonal_window, period=24, robust=True)
    result_temp = stl_temp.fit()
    
    sample = slice('2021-01-01', '2021-01-07')
    axes[i].plot(df.loc[sample].index, result_temp.seasonal[sample], 
                linewidth=2, label=f'seasonal={seasonal_window}')
    axes[i].set_title(f'Seasonal Component (window={seasonal_window})', fontweight='bold')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Decision Framework

![Decision Flowchart](decomposition_decision_flowchart.png)

### Quick Reference:

| Situation | Method | Why |
|-----------|--------|-----|
| Single, stable seasonality | Classical | Simple, fast |
| Single seasonality + outliers | STL | Robust |
| Multiple seasonal patterns | MSTL | Handles complexity |
| Evolving patterns | STL/MSTL | Adaptive |
| Missing data | STL | More tolerant |

## Checking Your Decomposition: Residual Diagnostics

Good residuals should be:
- Random (no patterns)
- Normally distributed
- Constant variance
- No autocorrelation

In [ ]:
from scipy import stats
import statsmodels.api as sm

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Time plot
axes[0, 0].plot(stl_result.resid)
axes[0, 0].set_title('Residuals Over Time')
axes[0, 0].axhline(y=0, color='r', linestyle='--')

# Histogram
axes[0, 1].hist(stl_result.resid.dropna(), bins=50, edgecolor='black')
axes[0, 1].set_title('Residual Distribution')
axes[0, 1].axvline(x=0, color='r', linestyle='--')

# Q-Q plot
sm.qqplot(stl_result.resid.dropna(), line='s', ax=axes[1, 0])
axes[1, 0].set_title('Q-Q Plot')

# ACF plot
sm.graphics.tsa.plot_acf(stl_result.resid.dropna(), lags=50, ax=axes[1, 1])
axes[1, 1].set_title('Autocorrelation')

plt.tight_layout()
plt.show()

print(f"Mean: {stl_result.resid.mean():.4f} (should be ~0)")
print(f"Std: {stl_result.resid.std():.4f}")
print(f"Skewness: {stats.skew(stl_result.resid.dropna()):.4f}")
print(f"Kurtosis: {stats.kurtosis(stl_result.resid.dropna()):.4f}")

## What's Next

Marcus went back with MSTL and nailed his Black Friday forecast within 5% error.

In **Part 3**, we tackle **Stationarity** - the concept that makes or breaks forecasting models.

You'll learn:
- Why non-stationary data breaks models
- How to test for stationarity (ADF, KPSS)
- Transformation techniques
- When to make data stationary vs. when not to

## Key Takeaways

- **Classical decomposition** breaks with multiple seasonalities or outliers
- **STL** is robust and adapts to changes
- **MSTL** handles multiple seasonal patterns
- Always **check residuals** to validate your decomposition
- Choose the right tool for your data complexity

---

*Next: Part 3 - Stationarity & Preprocessing*